# Imports

In [71]:
import pandas as pd
import re
import numpy as np
import glob
import tensorflow_hub as hub
import tensorflow_text as text

# Reading docs

In [72]:
def read_file(path, encoding, is_lower=False):
    list_of_paragraph = []
    buffer = []
    with open(path, encoding=encoding) as file:
        regex = re.compile(
            r'(((?<!Статья )(?<!^)(?<!(\.|\s))(?<!     )[а-яА-Я\d]{2,})([\.]{0,1}[\]\)\"]{0,2}[\.\;\:]{1}(\s|$)))',
            flags=re.IGNORECASE)
        text = file.read()

        if is_lower:
            text = text.lower()
        offset = 0
        buf = ''
        for ind, value in enumerate(regex.split(text, maxsplit=0)):
            if ind == 0 + offset:
                buf = value
            if ind == 1 + offset:
                list_of_paragraph.append((buf + value).strip())
                offset += 6

        second_regex = re.compile(r'([^\dгст][\.\;\:](?=\s|$))', flags=re.IGNORECASE)

        for value1 in list_of_paragraph:
            buf = ''
            for value in second_regex.split(value1, maxsplit=0):
                if value == '':
                    continue

                if len(value) > 2 and buf != '':
                    buffer.append(buf.strip())
                    buf = value
                elif len(value) > 2:
                    buf = value
                elif len(value) <= 2:
                    buffer.append((buf + value).strip())
                    buf = ''
    return buffer

### Getting file paths

In [73]:
list_of_path = list(filter(lambda x: not '1ДИ' in str(x), glob.glob("docs/*/*.txt")))
list_of_path

list_of_instruction_paths = list(filter(lambda x: str(x).startswith('docs\\Список ДИ'), list_of_path))
list_of_external_doc_paths = list(filter(lambda x: str(x).startswith('docs\\Список внешних'), list_of_path))

### Preparation of sentences

In [74]:
list_of_external_docs = []
list_of_instructions = []

in_lower_case = True

for path_to_doc in list_of_instruction_paths:
    buf_list = read_file(path_to_doc, 'windows-1251', in_lower_case)
    buf_list = [x for x in buf_list if len(x) > 3]
    list_of_instructions.append(buf_list)

for path_to_doc in list_of_external_doc_paths:
    buf_list = read_file(path_to_doc, 'windows-1251', in_lower_case)
    buf_list = [x for x in buf_list if len(x) > 3]
    list_of_external_docs.append(buf_list)

## Preparing the embeddings

### Download model

In [75]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

### Getting embeddings from instructions

In [76]:
%%time
BATCH_SIZE = 64
list_of_instruction_embeddings = []

for doc in list_of_instructions:
    buffer = np.empty((0, 512), float)
    for i in range(0, len(doc), BATCH_SIZE):
        batch = doc[i:i + BATCH_SIZE]
        buffer = np.concatenate((buffer, embed(batch)), axis=0)
        # buffer = np.concatenate((buffer, model.encode(batch)), axis=0)

    list_of_instruction_embeddings.append(buffer)

CPU times: total: 16.3 s
Wall time: 13.7 s


### Getting embeddings from external docs

In [77]:
%%time
BATCH_SIZE = 64
list_of_embeddings_of_external_docs = []

for doc in list_of_external_docs:
    buffer = np.empty((0, 512), float)
    for i in range(0, len(doc), BATCH_SIZE):
        batch = doc[i:i + BATCH_SIZE]
        buffer = np.concatenate((buffer, embed(batch)), axis=0)
        # buffer = np.concatenate((buffer, model.encode(batch)), axis=0)

    list_of_embeddings_of_external_docs.append(buffer)

CPU times: total: 15 s
Wall time: 12.4 s


### Computing similarity matrix

In [90]:
list_for_df = []

for index, doc1 in enumerate(list_of_instruction_embeddings):
    buffer = {
        'instruction_filename': list_of_instruction_paths[index]
    }
    for ind, doc2 in enumerate(list_of_embeddings_of_external_docs):
        buffer['external_doc_filename'] = list_of_external_doc_paths[ind]
        similarity_matrix = np.inner(doc1, doc2)
        for i, row in enumerate(similarity_matrix):
            buffer['sentence_number_from_instruction'] = i
            buffer['sentence_number_from_external_doc'] = int(np.argmax(row))
            buffer['value'] = np.max(row)
            buffer['text_of_instruction'] = list_of_instructions[index][i]
            buffer['text_of_external_doc'] = list_of_external_docs[ind][int(np.argmax(row))]
            list_for_df.append(buffer.copy())

## Creating dataframe

In [94]:
new_df = pd.DataFrame(
    columns=[
        'sentence_number_from_instruction',
        'sentence_number_from_external_doc',
        'value',
        'text_of_instruction',
        'instruction_filename',
        'text_of_external_doc',
        'external_doc_filename'
    ])

In [95]:
for index, row in enumerate(list_for_df):
    new_df.loc[index] = [
        row['sentence_number_from_instruction'],
        row['sentence_number_from_external_doc'],
        row['value'],
        row['text_of_instruction'],
        row['instruction_filename'],
        row['text_of_external_doc'],
        row['external_doc_filename'],
    ]

In [96]:
new_df

,sentence_number_from_instruction,sentence_number_from_external_doc,value,text_of_instruction,instruction_filename,text_of_external_doc,external_doc_filename
0,0,5,0.440365,общие положения\n 1.1. на должность дефектос...,docs\Список ДИ Юре 18.01.2023\ДИ деф-та УЗК.txt,1.5. к проведению сварочных и других огневых р...,docs\Список внешних документов 18.01.2023\Инст...
1,1,165,0.375417,"правил ростехнадзора, а также прошедших специа...",docs\Список ДИ Юре 18.01.2023\ДИ деф-та УЗК.txt,проверка знаний других специальных правил и ин...,docs\Список внешних документов 18.01.2023\Инст...
2,2,7,0.365666,1.2. один раз в з года дефектоскопист сдает к...,docs\Список ДИ Юре 18.01.2023\ДИ деф-та УЗК.txt,"1.6. персоналу, успешно прошедшему обучение и ...",docs\Список внешних документов 18.01.2023\Инст...
3,3,51,0.430704,1.3. дефектоскописты принимаются на работу ди...,docs\Список ДИ Юре 18.01.2023\ДИ деф-та УЗК.txt,3.4. наряд выдается с назначением ответственно...,docs\Список внешних документов 18.01.2023\Инст...
4,4,20,0.387139,прием на работу и увольнение производится в со...,docs\Список ДИ Юре 18.01.2023\ДИ деф-та УЗК.txt,"к работнику, грубо нарушившему правила проведе...",docs\Список внешних документов 18.01.2023\Инст...
...,...,...,...,...,...,...,...
85567,88,114,0.279474,6.1. за неисполнение всех возложенных на него ...,docs\Список ДИ Юре 18.01.2023\ДИ_Техник ТТЦ.txt,6. правительство российской федерации вправе у...,docs\Список внешних документов 18.01.2023\Феде...
85568,89,92,0.293590,"6.2. за нарушения требования птэ, птб, ппб, др...",docs\Список ДИ Юре 18.01.2023\ДИ_Техник ТТЦ.txt,5. право собственности и иные вещные права на ...,docs\Список внешних документов 18.01.2023\Феде...
85569,90,70,0.360748,"6.3. за невыполнение законодательных актов, пр...",docs\Список ДИ Юре 18.01.2023\ДИ_Техник ТТЦ.txt,3) иная установленная законом информация.,docs\Список внешних документов 18.01.2023\Феде...
85570,91,138,0.312496,6.4. за несоблюдение противопожарного режима.,docs\Список ДИ Юре 18.01.2023\ДИ_Техник ТТЦ.txt,6) постоянный контроль за обеспечением уровня ...,docs\Список внешних документов 18.01.2023\Феде...


In [97]:
with pd.ExcelWriter("embeddings.xlsx", engine="xlsxwriter") as writer:
    new_df.to_excel(writer, 'good', engine='xlsxwriter')
    sheets_good = writer.sheets['good']
    sheets_good.autofilter(0, 0, new_df.shape[0], new_df.shape[1])

    print("\nФайл создан")


Файл создан
